### **DAY 6 (14/01/26) – Medallion Architecture**

### Learn:

- Bronze (raw) → Silver (cleaned) → Gold (aggregated)
- Best practices for each layer
- Incremental processing patterns

### 🛠️ Tasks:

1. Design 3-layer architecture
2. Build Bronze: raw ingestion
3. Build Silver: cleaning & validation
4. Build Gold: business aggregates

### Architecture design overview:-

### BRONZE — Raw Ingestion Layer

**Purpose**:
Store raw CSV data exactly as received, plus ingestion metadata.

**Source**:
CSV files (Kaggle monthly dumps)

**Table**:
events_bronze

**Schema**:
event_time,\
event_type,\
product_id,\
category_id,\
category_code,\
brand,\
price,\
user_id,\
user_session,\
ingestion_ts   -- when the row entered the lakehouse

### SILVER — Clean & Trusted Layer
**Purpose**:
Create a clean, deduplicated, validated dataset for analytics and ML.

**Built From**:
events_bronze

**Table**:
events_silver

**Transformations**:
- Remove null / invalid rows
- Remove duplicates (latest per user_session + event_time + product_id)
- Filter valid event types (view, cart, purchase)
- Add derived columns

**Schema**:
event_time\
event_type\
product_id\
category_id\
category_code\
brand\
price\
user_id\
user_session\
event_date     -- DATE(event_time)\
price_tier     -- budget / mid / premium

### GOLD — Business Aggregates

**Purpose**:
Provide business-ready KPIs for dashboards and decision making.

**Built From**:
events_silver

**Table**:
events_gold_products

**Aggregations**:
- Views
- Carts
- Purchases
- Revenue
- Conversion rate
- Average order value

**Schema**:
product_id\
brand\
category_code\
views\
carts\
purchases\
revenue\
conversion_rate_pct\
avg_order_value.


In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True) #read data into df

from pyspark.sql import functions as F

events_bronze = df.withColumn("ingestion_ts", F.current_timestamp()) #added a timestamp column in our table format to see new entries with timestamps


In [0]:
# creating the delta table of november data in bronze architecture

events_bronze.write.format("delta").mode("overwrite").saveAsTable("events_bronze")

#events_bronze delta table created


In [0]:
%sql
CREATE OR REPLACE TABLE events_silver
USING DELTA
AS
SELECT
    -- business columns
    event_time, event_type, product_id, category_id, category_code, brand, price, user_id, user_session,

    -- enrichment
    DATE(event_time) AS event_date,

    CASE
        WHEN price < 10 THEN 'budget'
        WHEN price < 50 THEN 'mid'
        ELSE 'premium'
    END AS price_tier

FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY user_session, event_time, product_id
               ORDER BY ingestion_ts DESC
           ) AS rn
    FROM events_bronze --bronze schema being used to create filtered silver schema
    WHERE
        -- data quality filters
        event_time IS NOT NULL
        AND product_id IS NOT NULL
        AND user_id IS NOT NULL
        AND price IS NOT NULL
        AND price >= 0
        AND price < 10000
        AND event_type IN ('view', 'cart', 'purchase')
)
WHERE rn = 1; -- ensures “For every event, keep only the latest valid arrival.”


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE events_gold_products
USING DELTA
AS
SELECT
    product_id,
    brand,
    category_code,

    COUNT(DISTINCT CASE WHEN event_type = 'view' THEN user_id END)      AS views,
    COUNT(DISTINCT CASE WHEN event_type = 'cart' THEN user_id END)      AS carts,
    COUNT(DISTINCT CASE WHEN event_type = 'purchase' THEN user_id END) AS purchases,

    SUM(CASE WHEN event_type = 'purchase' THEN price END) AS revenue,

    ROUND(
        COUNT(DISTINCT CASE WHEN event_type = 'purchase' THEN user_id END)
        /
        NULLIF(COUNT(DISTINCT CASE WHEN event_type = 'view' THEN user_id END), 0)
        * 100, 2
    ) AS conversion_rate_pct,

    ROUND(
        SUM(CASE WHEN event_type = 'purchase' THEN price END)
        /
        NULLIF(COUNT(DISTINCT CASE WHEN event_type = 'purchase' THEN user_id END), 0)
    ,2) AS avg_order_value

FROM events_silver
GROUP BY
    product_id,
    brand,
    category_code;


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from events_bronze
limit 5;

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,ingestion_ts
2019-11-01T00:00:00.000Z,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,2026-01-14T04:17:09.739Z
2019-11-01T00:00:00.000Z,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283,2026-01-14T04:17:09.739Z
2019-11-01T00:00:01.000Z,view,17302664,2053013553853497655,null,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387,2026-01-14T04:17:09.739Z
2019-11-01T00:00:01.000Z,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,2026-01-14T04:17:09.739Z
2019-11-01T00:00:01.000Z,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2,2026-01-14T04:17:09.739Z


In [0]:
%sql
select * from events_silver
limit 5;

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,price_tier
2019-11-17T05:35:46.000Z,view,5100816,2053013553375346967,null,xiaomi,32.3,518709589,00000616-f016-4c01-b323-438486d9d3ee,2019-11-17,mid
2019-11-12T11:47:51.000Z,view,1005160,2053013555631882655,electronics.smartphone,xiaomi,203.07,556677167,0000264c-75d2-4049-ab5d-93e6dd64650d,2019-11-12,premium
2019-11-15T10:58:12.000Z,view,3700777,2053013565983425517,appliances.environment.vacuum,samsung,176.04,514504087,00002ea0-6a36-47a9-acc9-5b0024c586f9,2019-11-15,premium
2019-11-15T14:51:53.000Z,view,26600058,2053013563517174627,null,lucente,95.24,514010831,00006325-76f4-4a9c-b3ad-e0b28f9086ad,2019-11-15,premium
2019-11-15T14:56:42.000Z,view,26600082,2053013563517174627,null,lucente,120.72,514010831,00006325-76f4-4a9c-b3ad-e0b28f9086ad,2019-11-15,premium


In [0]:
%sql
select * from events_gold_products
limit 5;

product_id,brand,category_code,views,carts,purchases,revenue,conversion_rate_pct,avg_order_value
1801766,artel,electronics.video.tv,51942,3949,1828,340885.7299999999,3.52,186.48
26204573,null,null,105,1,1,441.4,0.95,441.4
28721601,respect,apparel.shoes,56,0,0,null,0.0,null
2501782,oursson,appliances.kitchen.oven,2442,229,99,13618.800000000001,4.05,137.56
43900033,matrix,null,794,27,10,558.4300000000001,1.26,55.84


## Conclusion:
Bronze → Raw CSV data stored in Delta with ingestion timestamp for traceability\
Silver → Cleaned, deduplicated, and validated events ready for analysis and ML\
Gold → Aggregated business metrics (revenue, conversions, performance) for dashboards and decisions